In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'Cancelled_Courses_1516_1617.xlsx',
 'Department_Allocations_20171.xlsx',
 'Description of Data.docx',
 'Description of Project.docx',
 'Marshall_Course_Enrollment_1516_1617.xlsx',
 'Marshall_Room_Capacity_Chart.xlsx',
 'seats_metric.ipynb',
 'Student_Course_Selection_1516.xlsx',
 'Summary_Special_Session_Codes_1516_1617.xlsx',
 '~$Marshall_Course_Enrollment_1516_1617.xlsx']

In [110]:
enroll = pd.read_excel('Marshall_Course_Enrollment_1516_1617.xlsx')
capacities = pd.read_excel('Marshall_Room_Capacity_Chart.xlsx')

In [111]:
capacities.head()

,Room,Size
0,ACC 306B,16
1,ACC201,48
2,ACC205,36
3,ACC236,39
4,ACC303,46


In [112]:
def convertTime(time):
    try:
        hh,mm,ss=time.split(':')
        ans=int(hh)*60+int(mm)+int(ss)/60
    except:
        ans=np.nan
    return ans

In [118]:
primeDict = {}

termSet = set(enroll.Term)
roomSet = set(capacities.Room)
dayList = ['M','T','W','H','F']
primeTimes = [x for x in range(480,1321,5)]

for term in termSet:
    for room in roomSet:
        for day in dayList:
            for time in primeTimes:
                key = (term,room,day,time)
                primeDict[key] = 0

In [119]:
for index,row in enroll.iterrows():
    
    term=row['Term']       
    room=row['First Room']
    days=row['First Days'] 
    reg=row['Reg Count']
    beg=convertTime(str(row['First Begin Time']))
    end=convertTime(str(row['First End Time']))
    
    if np.isnan(beg) or np.isnan(end) or room not in roomSet:  
        continue 
        
    for day in 'MTWHF':
        if day in days:
            for time in np.arange(beg,end+1,5):
                if time in primeTimes:
                    key = (term,room,day,time)
                    primeDict[key] = reg

In [120]:
lines = []
for term,room,day,time in primeDict:
    reg = primeDict[term,room,day,time]
    lines.append([term,room,day,time,reg])
output = pd.DataFrame(lines, columns = ['Term','Room','Day','Time','Registered'])

output.to_csv('room_usage.csv')

In [121]:
joined = output.merge(capacities, on='Room', how = 'left')
joined.head()

,Term,Room,Day,Time,Registered,Size
0,20161,ACC310,M,480,40,54
1,20161,ACC310,M,485,40,54
2,20161,ACC310,M,490,40,54
3,20161,ACC310,M,495,40,54
4,20161,ACC310,M,500,40,54


In [122]:
utils = joined.groupby(['Term','Day'])['Registered','Size'].sum() \
    .apply(lambda row: pd.Series({'Prime Util' : round(row.Registered / row.Size,2)}), axis = 1) \
    .reset_index()
    
utils['Day'] = pd.Categorical(utils['Day'], categories=['M','T','W','H','F'],ordered=True)
    
utils.pivot_table(index='Term', columns = 'Day', values = 'Prime Util')

Day,M,T,W,H,F
Term,,,,,
20153,0.32,0.29,0.31,0.27,0.11
20161,0.29,0.27,0.29,0.26,0.07
20162,0.07,0.07,0.07,0.07,0.03
20163,0.42,0.44,0.42,0.43,0.08
20171,0.41,0.42,0.44,0.40,0.10
20172,0.08,0.07,0.08,0.07,0.01
